In [1]:
"""
init
"""
import os
import io
import re
import ast
import pdb
import clip
import json
import torch
import faiss
import numpy as np
from PIL import Image
from tqdm import tqdm
Image.MAX_IMAGE_PIXELS = None
import matplotlib.pyplot as plt
from transformers import AutoModel, AutoTokenizer
from transformers import CLIPProcessor, CLIPModel

# init clip
clip_model, preprocess = clip.load("ViT-B/16", device="cuda")

# save index
def index_save(index_split, path):
    faiss.write_index(index_split, path)
    return

c:\Users\zhc23013\AppData\Local\miniconda3\envs\rag-ad\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
"""
build memory
"""
file_path = "C:/Users/zhc23013/Desktop/MLLM_RAG_COT/Memory/reference_image_locations.txt"
with open(file_path, 'r') as file:
    lines = file.readlines()
image_paths = [line.split()[0] for line in lines]
print(len(image_paths))

index_img_save_path =  "C:/Users/zhc23013/Desktop/MLLM_RAG_COT/Memory/memory.index"

index_img = faiss.IndexHNSWFlat(512, 64, faiss.METRIC_INNER_PRODUCT)

embed_img = []
with torch.no_grad():
    for image_path in tqdm(image_paths,desc="Process:"):
        image = preprocess(Image.open(image_path)).unsqueeze(0).to("cuda")
        image_features = clip_model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        embed_img.append(image_features.cpu())
embed_img = [np.array(embed) for embed in embed_img]
embed_img = np.array(embed_img).squeeze()
index_img.add(embed_img)
print("Total number of indexes:", index_img.ntotal)

print("Saving index:")
index_save(index_img, index_img_save_path)
print("Done")

12288


Process::   0%|          | 0/12288 [00:00<?, ?it/s]c:\Users\zhc23013\AppData\Local\miniconda3\envs\rag-ad\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
Process:: 100%|██████████| 12288/12288 [03:54<00:00, 52.40it/s]


Total number of indexes: 12288
Saving index:
Done


## retrieve 

In [3]:
index_img_save_path = "C:/Users/zhc23013/Desktop/MLLM_RAG_COT/Memory/memory.index"
index_img = faiss.read_index(index_img_save_path)

In [4]:
from PIL import Image
import torch
import numpy as np
from tqdm import tqdm

test_image_path = "C:/Users/zhc23013/Desktop/MLLM_RAG_COT/dataset/MMAD/DS-MVTec/capsule/image/crack/000.png"

def get_image_feature(image_path, clip_model, preprocess):
    image = preprocess(Image.open(image_path)).unsqueeze(0).to("cuda")
    with torch.no_grad():
        image_features = clip_model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)
    return image_features.cpu().numpy().squeeze()
    
test_image_feature = get_image_feature(test_image_path, clip_model, preprocess)


In [5]:
k = 1  
D, I = index_img.search(np.expand_dims(test_image_feature, axis=0), k)

print(f"Distance: {D}")
print(f"Index of the most similar image: {I[0][0]}")

most_similar_image_path = image_paths[I[0][0]]
print(f"Most similar image path: {most_similar_image_path}")

Distance: [[0.9959123]]
Index of the most similar image: 591
Most similar image path: C:\Users\zhc23013\Desktop\MLLM_RAG_COT\dataset\MMAD\MVTec-AD\capsule\train\good\158.png
